In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, classification_report, f1_score
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.display.max_rows = None

In [2]:
import numpy as np
print(np.__version__)

1.26.4


In [69]:
df = pd.read_csv("../Downloads/st_export.csv").iloc[:,1:]
for i in range(len(df)):
    df.iloc[i,-3]=df.iloc[i,-3].split("GMT")[0].strip()
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [91]:
a=np.random.randint(len(df))
df.iloc[a:a+5,:]

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
93446,100250536,-5.710986e+06,-48.566540,-0.793446,91256.523438,-0.002732,1.485665,1.187643,0.002019,-2.386726,-0.983239,BTC,2025-11-28 09:41:09,5m,60
93447,10624884,-1.438284e+05,-3.342660,-0.855477,896.125061,-0.000329,1.033427,1.525232,0.001660,-0.312593,-0.302795,BNB,2025-11-28 09:43:22,3m,100
93448,33493132,4.428978e+06,20.695633,0.707756,141.103104,-0.009660,0.793044,0.738676,0.005845,-0.968237,-0.747928,SOL,2025-11-28 09:43:22,3m,100
93449,22113486,-6.474776e+05,-15.281739,-0.826950,489.767273,-0.011306,1.152817,0.932625,0.009952,-1.221469,-0.840087,ZEC,2025-11-28 09:43:22,3m,100
93450,86119240,-2.493419e+06,-35.220108,-0.827435,3016.544922,-0.003502,1.352201,1.370381,0.004157,-1.561147,-0.915606,ETH,2025-11-28 09:43:22,3m,100


In [70]:
ndf = pd.read_csv("../Downloads/st_exportd.csv").iloc[:,1:]
for i in range(len(ndf)):
    ndf.iloc[i,-3]=ndf.iloc[i,-3].split("GMT")[0].strip()
ndf['timestamp'] = pd.to_datetime(ndf['timestamp'])
a=np.random.randint(len(ndf))
ndf.iloc[a:a+5,:]

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
34078,26488148,-1.586226e+06,-43.758087,-0.790513,987.358337,-0.008070,1.437581,1.852585,0.003905,-5.504299,-0.999967,BNB,2025-11-09 11:33:25,1m,300
34079,17821312,-3.278917e+06,-54.422829,-0.705622,2.269628,-0.010190,1.544228,1.517505,0.004154,-5.748526,-0.999980,XRP,2025-11-09 11:33:25,1m,300
34080,29636880,-4.824612e+05,-22.079174,-0.849464,104.872704,-0.043793,1.220792,0.332478,0.019967,-0.890205,-0.711495,LTC,2025-11-09 11:36:01,3m,100
34081,41716892,2.517887e+06,48.693920,0.782162,2.958139,-0.024049,0.513061,1.588092,0.021492,-0.911718,-0.721956,NEAR,2025-11-09 11:36:01,3m,100
34082,153388608,2.667445e+06,28.754623,0.845960,101907.125000,-0.003421,0.712454,1.200135,0.002613,-1.119427,-0.807370,BTC,2025-11-09 11:36:01,3m,100


In [88]:
nndf=ndf.iloc[df.tail(1).index.start+1:,:]
len(nndf)

617

In [100]:
assets_to_train=[i for i in nndf["asset"].unique().tolist() if i in df["asset"].unique().tolist()]
print(assets_to_train)

['SOL', 'BTC', 'XRP', 'ZEC', 'ETH', 'BNB', 'BCH', 'SUI', 'ZEN', 'TAO', 'NEAR', 'TON', 'DASH', 'RENDER', 'ORDI', 'WBTC', 'TRUMP', 'AAVE', 'INJ', 'LTC', 'FIL', 'DOT', 'LINK', 'CAKE', 'APT', 'NMR', 'AVAX', 'ETC', 'AR', 'GIGGLE', 'EGLD', 'ICP', 'UNI', 'PAXG']


In [101]:
c=[[len(df[df["asset"]==i]),i] for i in assets_to_train]

In [102]:
c = sorted(c, key=lambda x: x[0],reverse=True)
print(c)

[[11362, 'BTC'], [11336, 'ETH'], [10700, 'SOL'], [10692, 'ZEC'], [10346, 'XRP'], [10236, 'BNB'], [5212, 'SUI'], [3726, 'ICP'], [3340, 'DASH'], [3238, 'GIGGLE'], [3089, 'LTC'], [3089, 'LINK'], [2921, 'ZEN'], [2483, 'UNI'], [2437, 'NEAR'], [2379, 'TAO'], [2180, 'FIL'], [1952, 'TRUMP'], [1804, 'AVAX'], [1400, 'PAXG'], [1246, 'BCH'], [830, 'AAVE'], [692, 'AR'], [601, 'DOT'], [513, 'ETC'], [443, 'WBTC'], [393, 'CAKE'], [391, 'RENDER'], [381, 'ORDI'], [360, 'INJ'], [345, 'APT'], [322, 'TON'], [217, 'NMR'], [62, 'EGLD']]


# new

In [121]:
# train_and_evaluate_pipeline.py
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from datetime import timedelta

OUT_DIR = Path("./eval_outputs")
MODELS_DIR = OUT_DIR / "asset_models"
PRED_CSV = OUT_DIR / "predictions_nndf_with_metrics.csv"
PER_ASSET_CSV = OUT_DIR / "per_asset_metrics.csv"
PER_ASSET_GAP_CSV = OUT_DIR / "per_asset_gap_metrics.csv"

EVAL_MODE = "time_window"   # "first_k" or "time_window"
FIRST_K = 5                 # used if first_k mode
MIN_TRAIN_ROWS_PER_ASSET = 20  # warning if too small, but still attempts training

RANDOM_STATE = 42
N_ESTIMATORS = 200

# ---------- utility helpers ----------
def parse_gap_to_minutes(gap_str):
    # common formats: '1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','1w'
    if pd.isna(gap_str):
        return np.nan
    s = str(gap_str).strip().lower()
    if s.endswith('m'):
        return int(s[:-1])
    if s.endswith('M'):
        return int(s[:-1]) * 60 * 24 * 30
    if s.endswith('h'):
        return int(s[:-1]) * 60
    if s.endswith('d'):
        return int(s[:-1]) * 60 * 24
    if s.endswith('w'):
        return int(s[:-1]) * 60 * 24 * 7
    # fallback try int
    try:
        return int(s)
    except:
        return np.nan

def ensure_dirs():
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    MODELS_DIR.mkdir(parents=True, exist_ok=True)

# ---------- load & preprocess ----------
print("Loading CSVs...")
df_raw = df.copy()   # preserve your 'do not edit variable names' pattern if needed
ndf_raw = ndf.copy()

# parse timestamp columns
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
ndf_raw['timestamp'] = pd.to_datetime(ndf_raw['timestamp'])

# build nndf: rows in ndf after the last index row from df (your definition)
# (you used: nndf = ndf.iloc[df.tail(1).index.start+1:,:] — we replicate robustly)
last_df_index_in_ndf = None
# find the last (by timestamp) row in df and then select ndf rows with timestamp > that
last_df_ts = df_raw['timestamp'].max()
nndf = ndf_raw[ ndf_raw['timestamp'] > last_df_ts ].copy().reset_index(drop=True)
print(f"train_df rows: {len(df_raw):,}, full_df rows: {len(ndf_raw):,}, nndf rows: {len(nndf):,}")

# add gap_minutes and gaplimit (ensure gaplimit exists)
if 'gap' in df_raw.columns:
    df_raw['gap_min'] = df_raw['gap'].apply(parse_gap_to_minutes)
    nndf['gap_min'] = nndf['gap'].apply(parse_gap_to_minutes)
else:
    raise ValueError("No 'gap' column found in df — pipeline requires it.")

if 'gaplimit' not in df_raw.columns:
    df_raw['gaplimit'] = df_raw.get('gaplimit', np.nan)
if 'gaplimit' not in nndf.columns:
    nndf['gaplimit'] = nndf.get('gaplimit', np.nan)

# ---------- compute true_next_vwap for training ----------
# true_next_vwap is the next row's vwap within the same (asset, gap) group
print("Computing true_next_vwap in df (group by asset,gap sorted by timestamp)...")
df_sorted = df_raw.sort_values(['asset','gap','timestamp']).copy()
df_sorted['true_next_vwap'] = df_sorted.groupby(['asset','gap'])['vwap'].shift(-1)
train_df = df_sorted.dropna(subset=['true_next_vwap']).copy()  # rows that have a true next vwap
print("train_df rows (those with a valid next vwap):", len(train_df))

# ---------- feature selection ----------
# select numeric columns except target and meta columns:
meta_cols = set(['asset','timestamp','gap','gaplimit','gap_min','true_next_vwap'])
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
features = [c for c in numeric_cols if c not in meta_cols]
# ensure gap_min and gaplimit included
if 'gap_min' not in features:
    features.append('gap_min')
if 'gaplimit' not in features:
    features.append('gaplimit')
# dedupe
features = list(dict.fromkeys(features))
print("Using features:", features)

# ---------- training: per-asset model ----------
ensure_dirs()
assets_to_train = sorted(list(set(nndf['asset'].unique()).intersection(set(df_raw['asset'].unique()))))
print("Assets to forecast (intersection):", assets_to_train)

asset_models = {}
per_asset_train_metrics = []

for asset in assets_to_train:
    rows = train_df[ train_df['asset'] == asset ]
    if len(rows) < 1:
        print(f"--- Asset: {asset} -> no training rows (skip).")
        continue

    X = rows[features].fillna(0.0).values
    y = rows['true_next_vwap'].values

    # split for a quick validation
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.15, random_state=RANDOM_STATE)
    model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE, n_jobs=-1)
    model.fit(X_tr, y_tr)

    y_pred_val = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred_val)
    mae = mean_absolute_error(y_val, y_pred_val)
    r2 = r2_score(y_val, y_pred_val)

    per_asset_train_metrics.append({
        'asset': asset,
        'train_rows': len(rows),
        'val_RMSE': rmse,
        'val_MAE': mae,
        'val_R2': r2
    })

    model_path = MODELS_DIR / f"rf_{asset}.joblib"
    joblib.dump({'model':model, 'features':features}, model_path)
    asset_models[asset] = {'model':model, 'features':features, 'model_path':str(model_path)}
    print(f"--- Asset: {asset} --- Trained & saved -> {model_path}  (train rows: {len(rows)})")

per_asset_train_df = pd.DataFrame(per_asset_train_metrics)
per_asset_train_df.to_csv(OUT_DIR / "per_asset_train_metrics.csv", index=False)

# ---------- produce one prediction per (asset, gap) using the latest df row for that (asset,gap) ----------
print("\nProducing one prediction per (asset,gap) from latest df rows...")
pred_records = []

# define gaps we consider: union of gaps present in df for these assets (expected 11 per asset in ideal case)
gaps_universe = sorted(df_raw['gap'].dropna().unique().tolist(), key=lambda x: str(x))

for asset in assets_to_train:
    model_info = asset_models.get(asset)
    if model_info is None:
        print(f"Asset {asset} has no trained model. Skipping predictions for this asset.")
        continue
    model = model_info['model']
    feats = model_info['features']

    # for each gap available in the universe, try to create a prediction (one per gap)
    asset_df = df_raw[ (df_raw['asset']==asset) ].sort_values(['gap','timestamp'])
    # find last row per gap
    for gap in gaps_universe:
        sub = asset_df[ asset_df['gap'] == gap ]
        if sub.empty:
            # no last-row for this gap
            continue
        last_row = sub.sort_values('timestamp').iloc[-1]
        X_row = last_row[feats].fillna(0.0).values.reshape(1,-1)
        pred = model.predict(X_row)[0]

        pred_records.append({
            'asset': asset,
            'gap': gap,
            'gap_min': parse_gap_to_minutes(gap),
            'gaplimit': last_row.get('gaplimit', np.nan),
            'last_df_timestamp': last_row['timestamp'],
            'last_df_vwap': last_row['vwap'],
            'pred_next_vwap': float(pred),
            'model_path': model_info['model_path']
        })

pred_df = pd.DataFrame(pred_records)
print("Predictions produced:", len(pred_df))

# ---------- evaluation vs nndf ----------
print("\nEvaluating predictions against nndf using mode:", EVAL_MODE)
eval_rows = []

if len(nndf) == 0:
    print("nndf is empty. No evaluation possible. Predictions saved though.")
else:
    # make nndf indexed by asset & gap for quick selection
    nndf_sorted = nndf.sort_values('timestamp').copy()

    for _, prow in pred_df.iterrows():
        asset = prow['asset']
        gap = prow['gap']
        last_ts = prow['last_df_timestamp']
        gap_min = prow['gap_min']
        gaplimit = prow['gaplimit']
        pred_val = prow['pred_next_vwap']
        last_vwap = prow['last_df_vwap']

        # candidate true rows selection:
        cand = nndf_sorted[(nndf_sorted['asset']==asset) & (nndf_sorted['gap']==gap) & (nndf_sorted['timestamp'] > last_ts)]

        if EVAL_MODE == "first_k":
            cand_sel = cand.head(FIRST_K)
        else:  # time_window
            # window = gap_min * gaplimit (minutes) into the future
            # If gaplimit NaN, fall back to FIRST_K behavior
            if pd.isna(gap_min) or pd.isna(gaplimit):
                cand_sel = cand.head(FIRST_K)
            else:
                window_minutes = int(gap_min * gaplimit)
                window_end = last_ts + timedelta(minutes=window_minutes)
                cand_sel = cand[(cand['timestamp'] <= window_end)]

        if cand_sel.empty:
            eval_rows.append({
                'asset': asset, 'gap': gap, 'count_truths': 0,
                'pred_next_vwap': pred_val, 'rmse': np.nan, 'mae': np.nan, 'direction_acc': np.nan
            })
            continue

        truths = cand_sel['vwap'].values.astype(float)
        errors = truths - pred_val
        rmse = np.sqrt(np.mean(errors**2))
        mae = np.mean(np.abs(errors))
        # direction accuracy: compare sign(true - last_vwap) vs sign(pred - last_vwap)
        pred_dir = np.sign(pred_val - last_vwap)
        true_dirs = np.sign(truths - last_vwap)
        dir_acc = np.mean(true_dirs == pred_dir)

        eval_rows.append({
            'asset': asset,
            'gap': gap,
            'count_truths': len(truths),
            'pred_next_vwap': pred_val,
            'rmse': float(rmse),
            'mae': float(mae),
            'direction_acc': float(dir_acc),
            'last_df_timestamp': last_ts,
            'window_end_if_time_mode': (last_ts + timedelta(minutes=int(gap_min*gaplimit))).isoformat() if (EVAL_MODE=='time_window' and (not pd.isna(gap_min) and not pd.isna(gaplimit))) else None
        })

eval_df = pd.DataFrame(eval_rows)

# summarize per-asset metrics by averaging across gaps (weight by count_truths)
if not eval_df.empty:
    agg_records = []
    for asset, g in eval_df.groupby('asset'):
        valid = g.dropna(subset=['rmse','mae','direction_acc'])
        if valid.empty:
            agg_records.append({'asset':asset,'RMSE':np.nan,'MAE':np.nan,'Direction_Acc':np.nan,'Count':int(g['count_truths'].sum())})
            continue
        # weighted by counts where appropriate
        counts = valid['count_truths'].values
        rmse_w = np.average(valid['rmse'], weights=counts)
        mae_w = np.average(valid['mae'], weights=counts)
        diracc_w = np.average(valid['direction_acc'], weights=counts)
        agg_records.append({'asset':asset,'RMSE':float(rmse_w),'MAE':float(mae_w),'Direction_Acc':float(diracc_w),'Count':int(g['count_truths'].sum())})
    per_asset_metrics_df = pd.DataFrame(agg_records).sort_values('RMSE', ascending=False)
else:
    per_asset_metrics_df = pd.DataFrame(columns=['asset','RMSE','MAE','Direction_Acc','Count'])

# write outputs
ensure_dirs()
pred_full = pred_df.merge(eval_df, on=['asset','gap'], how='left', suffixes=('','_eval'))
pred_full.to_csv(PRED_CSV, index=False)
per_asset_metrics_df.to_csv(PER_ASSET_CSV, index=False)
eval_df.to_csv(PER_ASSET_GAP_CSV, index=False)

print("\nOutputs saved:")
print(" - Predictions + per-gap eval: ", PRED_CSV)
print(" - Per-asset metrics (aggregated): ", PER_ASSET_CSV)
print(" - Per-asset-gap eval table: ", PER_ASSET_GAP_CSV)
print("Pipeline complete.")

Loading CSVs...
train_df rows: 118,032, full_df rows: 118,649, nndf rows: 617
Computing true_next_vwap in df (group by asset,gap sorted by timestamp)...
train_df rows (those with a valid next vwap): 117408
Using features: ['klineacc', 'spread', 'spreadper', 'x', 'vwap', 'deviation', 'ratio', 'term', 'sigma', 'e', 'h', 'gap_min', 'gaplimit']
Assets to forecast (intersection): ['AAVE', 'APT', 'AR', 'AVAX', 'BCH', 'BNB', 'BTC', 'CAKE', 'DASH', 'DOT', 'EGLD', 'ETC', 'ETH', 'FIL', 'GIGGLE', 'ICP', 'INJ', 'LINK', 'LTC', 'NEAR', 'NMR', 'ORDI', 'PAXG', 'RENDER', 'SOL', 'SUI', 'TAO', 'TON', 'TRUMP', 'UNI', 'WBTC', 'XRP', 'ZEC', 'ZEN']
--- Asset: AAVE --- Trained & saved -> eval_outputs/asset_models/rf_AAVE.joblib  (train rows: 819)
--- Asset: APT --- Trained & saved -> eval_outputs/asset_models/rf_APT.joblib  (train rows: 335)
--- Asset: AR --- Trained & saved -> eval_outputs/asset_models/rf_AR.joblib  (train rows: 681)
--- Asset: AVAX --- Trained & saved -> eval_outputs/asset_models/rf_AVAX.jo

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_30503/2924291864.py:169: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_row = last_row[feats].fillna(0.0).values.reshape(1,-1)
/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_30503/2924291864.py:169: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_row = last_row[feats].fillna(0.0).values.reshape(1,-1)
/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_30503/2924291864.py:169: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will chang

Predictions produced: 369

Evaluating predictions against nndf using mode: time_window

Outputs saved:
 - Predictions + per-gap eval:  eval_outputs/predictions_nndf_with_metrics.csv
 - Per-asset metrics (aggregated):  eval_outputs/per_asset_metrics.csv
 - Per-asset-gap eval table:  eval_outputs/per_asset_gap_metrics.csv
Pipeline complete.


In [125]:
# ------------------- EVALUATION -------------------
from datetime import timedelta
import numpy as np
import pandas as pd

FIRST_K = 5  # fallback for missing gap_min/gaplimit

eval_rows = []

for _, prow in pred_df.iterrows():
    asset = prow['asset']
    gap = prow['gap']
    last_ts = prow['last_df_timestamp']
    gap_min = prow['gap_min']
    gaplimit = prow['gaplimit']
    pred_val = prow['pred_next_vwap']
    last_vwap = prow['last_df_vwap']

    # select candidate rows from existing nndf in memory
    cand = nndf[(nndf['asset'] == asset) &
                (nndf['gap'] == gap) &
                (nndf['timestamp'] > last_ts)]

    if pd.isna(gap_min) or pd.isna(gaplimit):
        cand_sel = cand.head(FIRST_K)
    else:
        window_end = last_ts + timedelta(minutes=int(gap_min * gaplimit))
        cand_sel = cand[cand['timestamp'] <= window_end]

    if cand_sel.empty:
        eval_rows.append({
            'asset': asset,
            'gap': gap,
            'count_truths': 0,
            'pred_next_vwap': pred_val,
            'rmse': np.nan,
            'mae': np.nan,
            'direction_acc': np.nan
        })
        continue

    truths = cand_sel['vwap'].values.astype(float)
    errors = truths - pred_val
    rmse = np.sqrt(np.mean(errors**2))
    mae = np.mean(np.abs(errors))

    # direction accuracy
    pred_dir = np.sign(pred_val - last_vwap)
    true_dirs = np.sign(truths - last_vwap)
    dir_acc = np.mean(true_dirs == pred_dir)

    eval_rows.append({
        'asset': asset,
        'gap': gap,
        'count_truths': len(truths),
        'pred_next_vwap': pred_val,
        'rmse': float(rmse),
        'mae': float(mae),
        'direction_acc': float(dir_acc)
    })

eval_df = pd.DataFrame(eval_rows)

# ------------------- AGGREGATE PER-ASSET -------------------
agg_records = []
for asset, g in eval_df.groupby('asset'):
    valid = g.dropna(subset=['rmse', 'mae', 'direction_acc'])
    if valid.empty:
        agg_records.append({
            'asset': asset,
            'RMSE': np.nan,
            'MAE': np.nan,
            'Direction_Acc': np.nan,
            'Count': int(g['count_truths'].sum())
        })
        continue
    counts = valid['count_truths'].values
    rmse_w = np.average(valid['rmse'], weights=counts)
    mae_w = np.average(valid['mae'], weights=counts)
    diracc_w = np.average(valid['direction_acc'], weights=counts)
    agg_records.append({
        'asset': asset,
        'RMSE': float(rmse_w),
        'MAE': float(mae_w),
        'Direction_Acc': float(diracc_w),
        'Count': int(g['count_truths'].sum())
    })

per_asset_metrics_df = pd.DataFrame(agg_records).sort_values('RMSE', ascending=False)

# ------------------- RESULTS -------------------
print("Per-prediction evaluation:")
display(eval_df)
print("\nPer-asset aggregated metrics:")
display(per_asset_metrics_df)


Per-prediction evaluation:


,asset,gap,count_truths,pred_next_vwap,rmse,mae,direction_acc
0,AAVE,12h,0,189.764409,NaN,NaN,NaN
1,AAVE,15m,0,158.793842,NaN,NaN,NaN
2,AAVE,1h,0,158.813456,NaN,NaN,NaN
3,AAVE,1m,0,157.904202,NaN,NaN,NaN
4,AAVE,2h,1,189.673788,0.093927,0.093927,1.000000
5,AAVE,30m,0,177.965491,NaN,NaN,NaN
6,AAVE,3m,0,158.135674,NaN,NaN,NaN
7,AAVE,4h,0,189.669511,NaN,NaN,NaN
8,AAVE,5m,0,166.398090,NaN,NaN,NaN
9,AAVE,6h,0,189.582642,NaN,NaN,NaN



Per-asset aggregated metrics:


,asset,RMSE,MAE,Direction_Acc,Count
6,BTC,233.770365,140.891108,0.321429,56
30,WBTC,39.842109,39.842109,1.000000,1
12,ETH,15.415644,10.447603,0.517857,56
32,ZEC,8.945182,7.442273,0.642857,56
26,TAO,2.180940,2.180940,1.000000,1
5,BNB,1.657783,0.862377,0.500000,56
4,BCH,0.792342,0.792342,1.000000,1
22,PAXG,0.767996,0.767996,1.000000,1
24,SOL,0.510154,0.263525,0.107143,56
18,LTC,0.174470,0.174470,0.000000,1
